<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/kedikopektespiti(cnn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Dogs vs Cats datasetini doğrudan indiriyoruz
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip


--2025-04-22 16:50:18--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.157.207, 142.251.8.207, 142.251.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.157.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  22.2MB/s    in 2.9s    

2025-04-22 16:50:21 (22.2 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [5]:
# Zip dosyasını çıkarıyoruz
import zipfile

with zipfile.ZipFile('cats_and_dogs_filtered.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [6]:
import os

train_dir = '/content/cats_and_dogs_filtered/train'
validation_dir = '/content/cats_and_dogs_filtered/validation'

# Veriyi kontrol edelim
print(f'Train directory: {os.listdir(train_dir)}')
print(f'Validation directory: {os.listdir(validation_dir)}')


Train directory: ['cats', 'dogs']
Validation directory: ['cats', 'dogs']


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Eğitim ve doğrulama için ImageDataGenerator oluşturma
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=32, class_mode='binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=32, class_mode='binary')


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [8]:
import tensorflow as tf

# CNN modeli oluşturma
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Kedi veya köpek için 1 çıktı
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modeli eğitme
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 89s 1s/step - accuracy: 0.5103 - loss: 0.8992 - val_accuracy: 0.5100 - val_loss: 0.6707
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.5512 - loss: 0.6766 - val_accuracy: 0.5350 - val_loss: 0.6840
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.5580 - loss: 0.6778 - val_accuracy: 0.6060 - val_loss: 0.6558
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.5884 - loss: 0.6588 - val_accuracy: 0.6030 - val_loss: 0.6672
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.6000 - loss: 0.6466 - val_accuracy: 0.6200 - val_loss: 0.6371
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.6009 - loss: 0.6581 - val_accuracy: 0.6370 - val_loss: 0.6489
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.6433 - loss: 0.6437 - val_accuracy: 0.6490 - val_loss: 0.6209
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.6826 - loss: 0.6046 - val_accuracy: 0.6780 - val_loss

In [15]:
# Modeli değerlendirme
test_loss, test_acc = model.evaluate(validation_generator)
print(f'Test accuracy: {test_acc}')

# Yeni bir görüntü üzerinde tahmin yapma
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = '/content/cats_and_dogs_filtered/train/dogs/dog.104.jpg'  # Test etmek istediğiniz resmin yolunu girin
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
print('Köpek' if prediction < 0.5 else 'Kedi')


32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 310ms/step - accuracy: 0.6851 - loss: 0.5837
Test accuracy: 0.6970000267028809
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Köpek
